* Karpathy video: https://www.youtube.com/watch?v=kCc8FmEb1nY
* Karpathy repo: https://github.com/karpathy/nanoGPT
* Colab for video: https://colab.research.google.com/drive/1JMLa53HDuA-i7ZBmqV7ZnA3c_fvtXnx-?usp=sharing

# Setup:

In [1]:
data_source = "maestro"

if data_source == "shakespeare":
    !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
else:
    !wget -N https://storage.googleapis.com/magentadata/datasets/maestro/v3.0.0/maestro-v3.0.0-midi.zip
    !unzip -n -qq maestro-v3.0.0-midi.zip

--2023-11-26 10:45:23--  https://storage.googleapis.com/magentadata/datasets/maestro/v3.0.0/maestro-v3.0.0-midi.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.136.207, 172.253.124.207, 142.250.105.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.136.207|:443... connected.
HTTP request sent, awaiting response... 304 Not Modified
File ‘maestro-v3.0.0-midi.zip’ not modified on server. Omitting download.



In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from glob import glob
import pathlib
import pretty_midi
from tqdm import tqdm
import numpy as np
import random 


In [3]:
use_wandb = True
if use_wandb:
    import wandb
    wandb.login()

wandb: Currently logged in as: drscotthawley. Use `wandb login --relogin` to force relogin


In [4]:
# Karpathy's # hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------


# Mine for Pitches
# hyperparameters
batch_size = 64 # how many independent sequences will we process in parallel?
block_size = 256 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
eval_iters = 200
n_embd = 128 # 64
n_head = 8 # 4
n_layer = 4 * 4
dropout = 0.1


# Mine for Full notes? 
# hyperparameters
batch_size = 96 # how many independent sequences will we process in parallel?
block_size = 256 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
#learning_rate = 1e-3
learning_rate = 3e-3
eval_iters = 200
n_embd = 128 # 64
n_head = 8 # 4
n_layer = 4 * 4
dropout = 0.1

# for comparison with giant_model code
batch_size = 128 # how many independent sequences will we process in parallel?
block_size = 128 # what is the maximum context length for predictions?
n_embd = 64 # 64  # dimensions of embeddings
n_head = 8 # 4
n_layer = 4 # # 4
dropout = 0.1 #0.1
max_iters = 25000
eval_interval = 100
eval_iters = 100
use_alibi = True


config = {
  'learning_rate': learning_rate,
  'batch_size': batch_size,
  'block_size': block_size,
  'n_embd': n_embd,
  'n_head': n_head,
  'n_layer': n_layer,
  'dropout': dropout
}

In [5]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

print('device =',device) 

device = cuda:0


# Dataset creation

In [6]:
notes_list = torch.load('maestro3_tensor_list_quant.pt')  # load from previous computation
#notes_list = torch.load('rastro-120bpm_16th_tensor_list.pt')  # load from previous computation
len(notes_list)

1276

In [7]:
def tl_to_notes(tensor_list, shuffle=False, delimit=True):
  "list of tensors (of arbitrary length, for each song) converted to one big long tensor of notes all running togehter"
  if shuffle:random.shuffle(tensor_list)
  if delimit:
    delimiter = torch.zeros(3)  # use all zeros to show ends of songs
    tensor_list = [element for item in tensor_list for element in (item, delimiter)]
  return torch.vstack(tensor_list).type(torch.float32)  # return one big tensor of floats

all_notes = tl_to_notes(notes_list, shuffle=True) 
all_pitches = all_notes[:,0].type(torch.long)  # just the pitch info
all_notes.shape, all_pitches.shape

(torch.Size([7041440, 3]), torch.Size([7041440]))

In [8]:
# @title Tensor to MIDI Display Code
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from typing import Optional
from IPython.display import Audio, display

def notes_arr_to_df(notes_arr) -> pd.DataFrame:
    columns = ['pitch','step','duration']
    df = pd.DataFrame(notes_arr, columns=columns)
    df["start"] = ""
    df["end"] = ""

    prev_start = 0
    #for i, row in tqdm(df.iterrows(), total=df.shape[0]):
    for i, row in df.iterrows():
        start = prev_start + float(row['step'])
        df.at[i, 'start'] = start
        df.at[i, 'end'] = start + float(row['duration'])
        prev_start = start
    return df

def df_to_midi(
        notes_df: pd.DataFrame,
        out_file: str = '',  # output file to save to, if any
        instrument_name: str = 'Acoustic Grand Piano', # whatever you want to call this instrument
        velocity: int = 100,  # note loudness
    ) -> pretty_midi.PrettyMIDI:
    "converts a dataframe to valid midi"

    pm = pretty_midi.PrettyMIDI()
    instrument = pretty_midi.Instrument(
        program=pretty_midi.instrument_name_to_program(
            instrument_name))

    prev_start = 0
    for i, note in notes_df.iterrows(): # this is a serial operation, not sure how to parallelize
        start = float(prev_start + note['step'])
        end = float(start + note['duration'])
        note = pretty_midi.Note(
            velocity=velocity,
            pitch=int(note['pitch']),
            start=start,
            end=end,
        )
        instrument.notes.append(note)
        prev_start = start

    pm.instruments.append(instrument)
    if out_file: pm.write(out_file)
    return pm

def plot_piano_roll(notes_df: pd.DataFrame, count: Optional[int] = None):
    "produce a piano roll plot"
    if count:
        title = f'First {count} notes'
    else:
        title = f'Whole track'
        count = len(notes_df['pitch'])
    plt.figure(figsize=(20, 4))
    plot_pitch = np.stack([notes_df['pitch'], notes_df['pitch']], axis=0)
    plot_start_stop = np.stack([notes_df['start'], notes_df['end']], axis=0)
    plt.plot(
        plot_start_stop[:, :count], plot_pitch[:, :count], color="b", marker=".")
    plt.xlabel('Time [s]')
    plt.ylabel('Pitch')
    ax = plt.gca()
    ax.set_ylim([0, vocab_size])
    _ = plt.title(title)
    plt.show()


def midi_to_audio(pm: pretty_midi.PrettyMIDI, seconds=30, sr=16000):
    "midi to audio, playable in notebook"
    waveform = pm.fluidsynth(fs=float(sr))
    # Take a sample of the generated waveform to mitigate kernel resets
    waveform_short = waveform[:seconds*sr] if seconds is not None else waveform
    return display(Audio(waveform_short, rate=sr))

def pitches_to_midi(pitch_list, seconds=30):
    notes_tensor = torch.zeros((len(pitch_list), 3)) + 0.25
    for i, p in enumerate(pitch_list):
        notes_tensor[i,0] = p
    notes_df = notes_arr_to_df(notes_tensor.cpu().detach().numpy())
    midi = df_to_midi(notes_df)
    plot_piano_roll(notes_df)
    audio_display = midi_to_audio(midi, seconds=seconds)
    return audio_display

def notes_to_midi(notes_tensor, seconds=30, time_rescale=None):
    notes_tensor = notes_tensor.clone() # just to avoid weird overwrites of memory
    #notes_tensor = notes_tensor * (notes_tensor>0)  # negative numbers clipped to zero
    if notes_tensor.min() < 0.0:
      print("WARNING: You have negative pitches, steps or durations. Setting them to zero")
      notes_tensor = notes_tensor * (notes_tensor >= 0)
    if time_rescale is not None :
        notes_tensor[:,1:] = notes_tensor[:,1:] *time_rescale # no quantization, just rescaling time
    notes_df = notes_arr_to_df(notes_tensor.cpu().detach().numpy())
    midi = df_to_midi(notes_df)
    plot_piano_roll(notes_df)
    audio_display = midi_to_audio(midi, seconds=seconds)
    return audio_display

In [9]:
all_notes = torch.load('jsb_tensor_rests.pt') 
all_pitches = all_notes[:,0].type(torch.long)  # just the pitch info
all_notes.shape, all_pitches.shape

(torch.Size([78800, 3]), torch.Size([78800]))

## Codebook creation

In [10]:
#chars = list(text)
# data here.
#raw_data = torch.tensor([ord(c) for c in chars],dtype=torch.int32).unsqueeze(1) # sadly torch has no char dtype
#raw_data = np.array(list(all_pitches))[:,np.newaxis] # but numpy does
raw_data = all_pitches.unsqueeze(-1)
raw_data = all_notes
print("raw_data.shape =",raw_data.shape)
n_codebooks = raw_data.shape[-1]
print("n_codebooks =",n_codebooks)
codebooks = []
for i in range(n_codebooks): 
    #cb_vals = sorted(set(raw_data[:,i]))
    cb_vals = raw_data.unique().sort()[0] if i>0 else torch.arange(128)  # use all possible pitches 
    #print(f"cb {i}: cb_vals = {cb_vals}")
    codebooks.append({'encode':{k.item(): int(v) for v, k in enumerate(cb_vals)}, 
                      'decode':{int(v): k for v, k in enumerate(cb_vals)}})
    print(f"--- cb {i}: cb keys = {codebooks[-1]['encode'].keys()}")
vocab_sizes = [len(cb['encode']) for cb in codebooks]
print("vocab_sizes = ",vocab_sizes)
vocab_size = vocab_sizes[0]
vocab_size

raw_data.shape = torch.Size([78800, 3])
n_codebooks = 3
--- cb 0: cb keys = dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127])
--- cb 1: cb keys = dict_keys([0.0, 0.11999999731779099, 0.23999999463558197, 0.36000001430511475, 0.47999998927116394, 0.6000000238418579, 0.7200000286102295, 0.8399999737739563, 0.9599999785423279, 1.0800000429153442, 1.2000000476837158, 1.3200000524520874, 1.440000057220459, 1.559999942779541, 1.6799999475479126, 1.7999999523162842, 1.9199999570846558, 2.0399999618530273, 2.16

128

Encoder and decoder

In [11]:
#these are slow and could be improved a lot by doing all cbs at once
#encode = lambda s: torch.tensor( [[codebooks[cb]['encode'][note[cb].item()] for cb in range(n_codebooks)] for note in s], dtype=torch.long).to(s.device)
        
#decode = lambda si: torch.tensor([[codebooks[cb]['decode'][i[cb].item()]  for cb in range(n_codebooks) ] for i in si]).to(si.device)

def remap_vals(s, direction, dtype=torch.long):
    out = torch.zeros_like(s, dtype=dtype)
    for cb in range(s.shape[-1]): 
        dict_map = codebooks[cb][direction]
        out[:,cb] = torch.tensor([dict_map[x.item()] for x in s[:,cb]], dtype=dtype)
    return out 

encode = lambda s: remap_vals(s, 'encode')
decode = lambda s: remap_vals(s, 'decode', dtype=all_notes.dtype)


#test_str = 'hello!'
#test_str = [torch.tensor(q) for q in [66,32,88,0,98]]
test_str = all_notes[0:6]
ilist = encode(test_str)
print("test_str =",test_str)
ret_str = decode(ilist)
assert torch.equal(test_str, ret_str), f"Oops. test_str={test_str}, but ret_str={ret_str}"
print("Checks pass! :-)")

test_str = tensor([[127.0000,   0.0000,   0.9600],
        [ 58.0000,   0.9600,   0.7200],
        [ 65.0000,   0.0000,   0.4800],
        [ 70.0000,   0.0000,   0.9600],
        [ 74.0000,   0.0000,   0.4800],
        [ 55.0000,   0.4800,   0.4800]])
Checks pass! :-)


In [12]:
data = encode(all_notes) # this takes a while
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]
print("train_data.shape, val_data.shape =",train_data.shape, val_data.shape)

train_data.shape, val_data.shape = torch.Size([70920, 3]) torch.Size([7880, 3])


In [13]:
torch.manual_seed(1337)


# def augment_data(x,y): # db = datablock - a seqeunce , likely of length 1+block_size 
#     x,y = x.clone(), y.clone()  # avoid in-place alterations of data
#     pitch_shift = torch.randint(low=-6, high=6, size=(1,1,1), dtype=torch.long).item()
#     x[::,0] = torch.clamp(x[::,0] + pitch_shift, 0, vocab_sizes[0]-1) 
#     y[::,0] = torch.clamp(y[::,0] + pitch_shift, 0, vocab_sizes[0]-1) 
#     return x,y


def augment_data_db(db, do_time=False, pb=12): # db = datablock - a seqeunce , likely of length 1+block_size 
    db = db.clone()  # avoid in-place alterations of data
    #pitch_shift = torch.randint(low=-pb, high=pb, size=(1,1,1), dtype=torch.long).item()
    db[db[:,:,0] < 127] += random.randint(-pb, pb)
    return db  # this is sufficient
    if False and do_time: # doesn't work. 
        time_scale = torch.randint(1, 2, (1,)) 
        db[:,:,1] = torch.where( db[:,:,0] < 127,  db[:,:,1]+pitch_shift,  db[:,:,1])
        db[:,:,2] = torch.where( db[:,:,0] < 127,  db[:,:,2]+pitch_shift,  db[:,:,2])

        
    for i, vs in enumerate(vocab_sizes): # stay in bounds
        print("i = ",i," db before clamp = \n",db)
        db[:,:,i] = torch.clamp(db[:,:,i], 0, vs-1)
        print("i = ",i," db after clamp = \n",db)
    return db

In [14]:
# test data augmentation
db = torch.tensor([[54,12,6],[61,0,124],[127,31,4],[86,0,12],[126,7,12]]).unsqueeze(0)
print("original db = \n",db)
ret = augment_data_db(db) # make sure the note with the "127" rest is left completely alone
print(ret)
assert torch.equal(db[0,2,:], ret[0,2,:]),"Those lines don't match"
print("All good!")

original db = 
 tensor([[[ 54,  12,   6],
         [ 61,   0, 124],
         [127,  31,   4],
         [ 86,   0,  12],
         [126,   7,  12]]])
tensor([[[ 66,  24,  18],
         [ 73,  12, 136],
         [127,  31,   4],
         [ 98,  12,  24],
         [138,  19,  24]]])
All good!


In [15]:
# data loading
def get_batch(split, debug=False):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,)) # batch_size number of indices to grab from
    data_block = torch.stack([data[i:i+block_size+1] for i in ix])
    if split=='train': 
        data_block = augment_data_db(data_block)
    x, y = data_block[:,:-1,:], data_block[:,1:,:]
    #x = torch.stack([data[i:i+block_size] for i in ix])
    #y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    #x, y = augment_data(x,y)
    x, y = x.to(device), y.to(device)
    if debug: print(f"get_batch: x.shape = {x.shape}, y.shape = {y.shape}")
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [16]:
#%%timeit 
#x,y = get_batch('val')
# output: 1.55 ms ± 916 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)

In [17]:
x,y = get_batch('train',debug=True) # first time is slow
print(f"B, T = {batch_size}, {block_size}")

get_batch: x.shape = torch.Size([128, 128, 3]), y.shape = torch.Size([128, 128, 3])
B, T = 128, 128


x is a sequence

In [18]:
x[0]  # .T is to show it horizontal instead of vertical

tensor([[ 43,  -9,  -9],
        [ 46, -11,  -9],
        [ 63, -11,  -7],
        [ 45,  -9,  -9],
        [ 48, -11,  -9],
        [ 60, -11,  -7],
        [ 46,  -9,  -7],
        [ 49, -11,  -9],
        [ 61, -11,  -7],
        [ 53,  -9,  -9],
        [ 58, -11,  -5],
        [ 39,  -9,  -7],
        [ 54, -11,  -7],
        [ 60, -11,  -5],
        [ 41,  -7,  -7],
        [ 53, -11,  -9],
        [ 57, -11,  -7],
        [ 51,  -9,  -9],
        [ 58, -11,  10],
        [ 34,  -9,  -3],
        [ 50, -11,  -1],
        [ 53, -11,  -3],
        [ 46,  -3,  -9],
        [ 65, -11,  -3],
        [ 48,  -9,  -9],
        [ 51, -11,  -9],
        [ 50,  -9,  -7],
        [ 53, -11,  -9],
        [ 46,  -9,  -9],
        [ 51,  -9,   1],
        [ 66, -11,  -3],
        [ 49,  -9,  -9],
        [ 47,  -9,  -9],
        [ 59, -11,  -3],
        [ 46,  -9,  -9],
        [ 44,  -9,  -9],
        [ 65, -11,  -3],
        [ 41,  -9,  -9],
        [ 46,  -9,  -7],
        [ 50, -11,  -7],


y is x shifted back by one and including new data.
in this sense only y[:,-1] is the "next token" being predicted.

In [19]:
y[0]

tensor([[ 46, -11,  -9],
        [ 63, -11,  -7],
        [ 45,  -9,  -9],
        [ 48, -11,  -9],
        [ 60, -11,  -7],
        [ 46,  -9,  -7],
        [ 49, -11,  -9],
        [ 61, -11,  -7],
        [ 53,  -9,  -9],
        [ 58, -11,  -5],
        [ 39,  -9,  -7],
        [ 54, -11,  -7],
        [ 60, -11,  -5],
        [ 41,  -7,  -7],
        [ 53, -11,  -9],
        [ 57, -11,  -7],
        [ 51,  -9,  -9],
        [ 58, -11,  10],
        [ 34,  -9,  -3],
        [ 50, -11,  -1],
        [ 53, -11,  -3],
        [ 46,  -3,  -9],
        [ 65, -11,  -3],
        [ 48,  -9,  -9],
        [ 51, -11,  -9],
        [ 50,  -9,  -7],
        [ 53, -11,  -9],
        [ 46,  -9,  -9],
        [ 51,  -9,   1],
        [ 66, -11,  -3],
        [ 49,  -9,  -9],
        [ 47,  -9,  -9],
        [ 59, -11,  -3],
        [ 46,  -9,  -9],
        [ 44,  -9,  -9],
        [ 65, -11,  -3],
        [ 41,  -9,  -9],
        [ 46,  -9,  -7],
        [ 50, -11,  -7],
        [ 58, -11,  -9],


# Model definition

In [20]:

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self, debug=False):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        print("vocab sizes = ",[vocab_sizes[cb] for cb in range(n_codebooks)])
        self.token_embedding_tables = nn.ModuleList([nn.Embedding(vocab_sizes[cb], n_embd) for cb in range(n_codebooks)])
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_heads = nn.ModuleList([nn.Linear(n_embd, vocab_sizes[cb]) for cb in range(n_codebooks)])
        self.debug=debug

    def forward(self, idx, targets=None):
        B, T, CBS = idx.shape
        tok_emb = 0
        for cb in range(CBS): # just sum codebook reps
            tok_emb = tok_emb + self.token_embedding_tables[cb](idx[:,:,cb])
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,E)

        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits_list = [head(x) for head in self.lm_heads]  # list of (B,T,vocab_sizes) tensors, CBS long

        if targets is None:
            loss = None
        else:
            lambdas = [0.5]*CBS
            loss = 0.0
            for cb in range(CBS):  # loop over codebooks
                logits = logits_list[cb]  
                B, T, V = logits.shape   # V = vocab size
                targ = targets[:,:,cb]   # B, T 
                logits = logits.view(B*T, V)
                targ = targ.reshape(B*T)
                loss = loss + lambdas[cb]*F.cross_entropy(logits, targ)
        if self.debug: print("loss = ",loss.item()) 
        return logits_list, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:, :]
            # get the predictions
            logits_list, loss = self(idx_cond)
            idx_next_list = []
            for cb in range(ind_cond.shape[-1]):
                # focus only on the last time step
                logits = logitslist[cb][:, -1] # becomes (B, C)
                # apply softmax to get probabilities
                probs = F.softmax(logits, dim=-1) # (B, C)
                # sample from the distribution
                idx_next_list.append(torch.multinomial(probs, num_samples=1)) # (B, 1)
            idx_next = torch.tensor(idx_next_list).to(idx.device)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx



Instantiate and get ready to run

In [21]:
model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

vocab sizes =  [128, 93, 93]
0.247994 M parameters


In [22]:
def save_checkpoint(step, model, optimizer,loss, name):
    name = name + f'_{step}.pt'
    print("Saving checkpoint to",name)
    torch.save({
            'step': step,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
            }, name)

def load_checkpoint(name, model, optimizer):
    checkpoint = torch.load(name)
    model.load_state_dict(checkpoint['model_state_dict'])
    m = model.to(device)
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    step = checkpoint['step']
    loss = checkpoint['loss']
    return step, m, optimizer, loss

# Do training

In [23]:
if use_wandb: wandb.init(project='musicbox-jsb', config=config)

In [24]:
for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
        if use_wandb: wandb.log(losses | {'step':iter//eval_interval})

    
    xb, yb = get_batch('train') # sample a batch of data
    logits, loss = model(xb, yb) # evaluate the loss
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()


../aten/src/ATen/native/cuda/Indexing.cu:1292: indexSelectLargeIndex: block: [111,0,0], thread: [32,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1292: indexSelectLargeIndex: block: [111,0,0], thread: [33,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1292: indexSelectLargeIndex: block: [111,0,0], thread: [34,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1292: indexSelectLargeIndex: block: [111,0,0], thread: [35,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1292: indexSelectLargeIndex: block: [111,0,0], thread: [36,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1292: indexSelectLargeIndex: block: [111,0,0], thread: [37,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1292: indexSelectLargeIndex: block: [111,

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
save_checkpoint(max_iters, model, optimizer,loss, f"fullpitches_ncbs_jsb")

In [ ]:
if use_wandb: wandb.finish()

Generate

In [ ]:

def generator(model, idx, max_new_tokens, temperature=1.0):
    # idx is (B, T, CBS) array of indices in the current context
    for _ in range(max_new_tokens):
        # crop idx to the last block_size tokens
        idx_cond = idx[:, -block_size:]
        # get the predictions
        logits_list, loss = model(idx_cond)
        idx_next_list = []
        for cb in range(idx_cond.shape[-1]):
            # focus only on the last time step
            logits = logits_list[cb][:, -1, :] # becomes (B, V)
            # apply softmax to get probabilities
            probs = F.softmax(logits/temperature, dim=-1) # (B, V)
            # sample from the distribution
            idx_next_list.append(torch.multinomial(probs, num_samples=1)) # (B, 1)
        idx_next = torch.tensor(idx_next_list).unsqueeze(0).unsqueeze(0).to(idx.device)
        # append sampled index to the running sequence
        
        idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
    return idx


In [ ]:
ind, num = 2000, 128
context = all_notes[ind:ind+num].clone()
#context[:,0] = context[:,0]
context = encode(context).to(device)
len(context)

In [ ]:
time_rescale = 1.0
notes_to_midi(decode(context), time_rescale=time_rescale)

In [ ]:
#context = torch.zeros((1, 3), dtype=torch.long, device=device)
notes = decode( generator(model, context.unsqueeze(0), max_new_tokens=500, temperature=0.5)[0].cpu() )
len(notes)

In [ ]:
time_rescale = 1.0
notes_to_midi(notes, time_rescale=time_rescale, seconds=None)